# Getting the Kaggle key




In [ ]:
# Upload your kaggle.json api key
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"henrilemoine","key":"8ceb16864369510e55ed40be36859fc3"}'}

# Getting the Dataset

In [ ]:
%%capture
!pip install -q kaggle

# Create Kaggle folder
! mkdir ~/.kaggle
# copy the kaggle.json to folder created
! cp kaggle.json ~/.kaggle/

# permission for the json to act
! chmod 600 ~/.kaggle/kaggle.json

# to list all datasets in kaggle
! kaggle datasets list

! kaggle datasets download -d konradb/atticus-open-contract-dataset-aok-beta

!unzip atticus-open-contract-dataset-aok-beta.zip

In [ ]:
import csv
import pandas as pd

master_clause_df = pd.read_csv("Final Publication/master_clauses.csv")
master_clause_df = master_clause_df.dropna(axis=1)
print(master_clause_df)
filenames = master_clause_df.Filename.to_list()
contract = []
for f in filenames:
    contract_type = f.split("_")[-1].split(".")[0].strip()
    if contract_type[0].isnumeric():
        contract_type = "NaN"
    if contract_type[-1].isnumeric():
        contract_type = contract_type[:-1]
    if contract_type == 'WESTERN COPPER - NON-COMPETITION AGREEMENT':
        contract_type = "NaN"
    if contract_type == 'VIVINT SOLAR, INC':
        contract_type = "NaN"
    if contract_type == 'Quaker Chemical Corporation - NON COMPETITION AND NON SOLICITATION AGREEMENT':
        contract_type = "NaN"
  
    contract.append(contract_type)
    
master_clause_df["Contract Type"] = contract

                                              Filename  ... Third Party Beneficiary-Answer
0    CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...  ...                             No
1    EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...  ...                             No
2    FulucaiProductionsLtd_20131223_10-Q_EX-10.9_83...  ...                             No
3    GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10...  ...                             No
4    IdeanomicsInc_20160330_10-K_EX-10.26_9512211_E...  ...                             No
..                                                 ...  ...                            ...
195  BORROWMONEYCOM,INC_06_11_2020-EX-10.1-JOINT VE...  ...                             No
196  VEONEER,INC_02_21_2020-EX-10.11-JOINT VENTURE ...  ...                             No
197  KIROMICBIOPHARMA,INC_04_08_2020-EX-10.28-JOINT...  ...                             No
198  MJBIOTECH,INC_12_06_2018-EX-99.01-JOINT VENTUR...  ...                             No

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
%%capture
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased').to('cuda')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from os import walk

mypath = "Final Publication/individual_contract_clauses"
filenames = next(walk(mypath), (None, None, []))[2]  # [] if no file
num_files = len(filenames)

In [ ]:
all_embedings = []
for i, f in enumerate(filenames[:38]):
    print(f"[{i}/{num_files}] {f}")
    temp_df = pd.read_csv(f"Final Publication/individual_contract_clauses/{f}", error_bad_lines=False)
    text = temp_df["Clause"].to_list()
    states = []
    for line in text:
        encoding = tokenizer(line[:100], return_tensors = "pt").to('cuda')
        outputs = model(**encoding)
        states.append(outputs['last_hidden_state'][0][0].to('cpu'))
    p = torch.stack(states, dim=0).to('cpu')
    avg = torch.mean(p, 0)
    all_embedings.append(avg)

[0/200] BellringBrandsInc_20190920_S-1_EX-10.12_11817081_EX-10.12_Manufacturing Agreement1.csv
[1/200] MOELIS_CO_03_24_2014-EX-10.19-STRATEGIC ALLIANCE AGREEMENT.csv
[2/200] RandWorldwideInc_20010402_8-KA_EX-10.2_2102464_EX-10.2_Co-Branding Agreement.csv
[3/200] LegacyEducationAllianceInc_20141110_8-K_EX-10.9_8828866_EX-10.9_Endorsement Agreement.csv
[4/200] MphaseTechnologiesInc_20030911_10-K_EX-10.15_1560667_EX-10.15_Co-Branding Agreement.csv
[5/200] CardlyticsInc_20180112_S-1_EX-10.16_11002987_EX-10.16_Maintenance Agreement4.csv
[6/200] LeadersonlineInc_20000427_S-1A_EX-10.8_4991089_EX-10.8_Co-Branding Agreement.csv
[7/200] GALERATHERAPEUTICS,INC_02_14_2020-EX-99.A-JOINT FILING AGREEMENT.csv
[8/200] PharmagenInc_20120803_8-KA_EX-10.1_7693204_EX-10.1_Endorsement Agreement.csv
[9/200] GpaqAcquisitionHoldingsInc_20200123_S-4A_EX-10.6_11951677_EX-10.6_License Agreement.csv
[10/200] PLAYAHOTELS_RESORTSNV_03_14_2017-EX-10.22-STRATEGIC ALLIANCE AGREEMENT (Hyatt Ziva Cancun).csv
[11/200] Pa

b'Skipping line 6: expected 5 fields, saw 7\n'


[19/200] GpaqAcquisitionHoldingsInc_20200123_S-4A_EX-10.8_11951679_EX-10.8_Service Agreement.csv
[20/200] LinkPlusCorp_20050802_8-K_EX-10_3240252_EX-10_Affiliate Agreement.csv
[21/200] BORROWMONEYCOM,INC_06_11_2020-EX-10.1-JOINT VENTURE AGREEMENT.csv
[22/200] SoupmanInc_20150814_8-K_EX-10.1_9230148_EX-10.1_Franchise Agreement1.csv


b'Skipping line 8: expected 5 fields, saw 7\n'


[23/200] GluMobileInc_20070319_S-1A_EX-10.09_436630_EX-10.09_Content License Agreement3.csv
[24/200] ReedsInc_20191113_10-Q_EX-10.4_11888303_EX-10.4_Development Agreement.csv
[25/200] AzulSa_20170303_F-1A_EX-10.3_9943903_EX-10.3_Maintenance Agreement2.csv
[26/200] WestPharmaceuticalServicesInc_20200116_8-K_EX-10.1_11947529_EX-10.1_Supply Agreement.csv
[27/200] TheglobeComInc_19990503_S-1A_EX-10.20_5416126_EX-10.20_Co-Branding Agreement.csv
[28/200] LegacyEducationAllianceInc_20200330_10-K_EX-10.18_12090678_EX-10.18_Development Agreement.csv
[29/200] SmartRxSystemsInc_20180914_1-A_EX1A-6 MAT CTRCT_11351705_EX1A-6 MAT CTRCT_Distributor Agreement.csv


b'Skipping line 5: expected 5 fields, saw 7\n'


[30/200] VALENCETECHNOLOGYINC_02_14_2003-EX-10-JOINT VENTURE CONTRACT.csv
[31/200] PcquoteComInc_19990721_S-1A_EX-10.11_6377149_EX-10.11_Co-Branding Agreement1.csv
[32/200] ImpresseCorp_20000322_S-1A_EX-10.11_5199234_EX-10.11_Co-Branding Agreement.csv
[33/200] ScansourceInc_20190822_10-K_EX-10.38_11793958_EX-10.38_Distributor Agreement2.csv
[34/200] ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement.csv


b'Skipping line 10: expected 7 fields, saw 9\n'


[35/200] GridironBionutrientsInc_20171206_8-K_EX-10.2_10972556_EX-10.2_Endorsement Agreement.csv
[36/200] DigitalCinemaDestinationsCorp_20111220_S-1_EX-10.10_7346719_EX-10.10_Affiliate Agreement.csv
[37/200] ImperialGardenResortInc_20161028_DRS (on F-1)_EX-10.13_9963189_EX-10.13_Outsourcing Agreement.csv


In [ ]:
question = "Change of Control-Answer"

X = []
for i in all_embedings:
    X.append(i.detach().numpy())
y = []
for i in master_clause_df[question][:30].to_list():
    if i == "No":
        y.append(0)
    elif i == "Yes":
        y.append(1)
    else:
        raise Error
prc = 68
n = 30
X_train, X_test = X[:int(prc/100*n)], X[int(prc/100*n):n]
y_train, y_test = y[:int(prc/100*n)], y[int(prc/100*n):n]
print(y_train, y_test)

[0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0] [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression(n_jobs=5)
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=5, normalize=False)

In [ ]:
print("y | pred")
for i in range(len(X_test)):
    prediction = regr.predict(X_test[i].reshape(1, 768))
    print(f"{y_test[i]} -> {np.rint(prediction)[0]:.0f} ({prediction[0]:.4f})")

y | pred
0 -> 1 (0.8233)
0 -> 0 (0.4470)
0 -> 0 (0.1280)
0 -> 1 (0.9336)
0 -> 0 (0.1924)
0 -> -0 (-0.2365)
0 -> 0 (0.2723)
0 -> 1 (0.6722)
1 -> 0 (0.3735)
1 -> 0 (0.4973)


In [ ]:
class SimpleLinearRegression:
    '''
    A class which implements simple linear regression model.
    '''
    def __init__(self):
        self.b0 = None
        self.b1 = None
    
    def fit(self, X, y):
        '''
        Used to calculate slope and intercept coefficients.
        
        :param X: array, single feature
        :param y: array, true values
        :return: None
        '''
        numerator = np.sum((X - np.mean(X)) * (y - np.mean(y)))
        denominator = np.sum((X - np.mean(X)) ** 2)
        self.b1 = numerator / denominator
        self.b0 = np.mean(y) - self.b1 * np.mean(X)
        
    def predict(self, X):
        '''
        Makes predictions using the simple line equation.
        
        :param X: array, single feature
        :return: None
        '''
        if not self.b0 or not self.b1:
            raise Exception('Please call `SimpleLinearRegression.fit(X, y)` before making predictions.')
        return self.b0 + self.b1 * X

In [ ]:
linear_regression_model = SimpleLinearRegression()
linear_regression_model.fit(X_train, y_train)
preds = linear_regression_model.predict(X_test)

linear_regression_model.b0, linear_regression_model.b1

In [ ]:

class Bert_Model(nn.Module):
   def __init__(self, class):
       super(Bert_Model, self).__init__()
       self.bert = BertModel.from_pretrained('bert-base-uncased')
       self.out = nn.Linear(self.bert.config.hidden_size, classes)
   def forward(self, input):
       _, output = self.bert(**input)
       out = self.out(output)
       return out